In [2]:
import pandas as pd
import numpy as np
import os
from scipy import stats
pd.options.display.max_columns = None


In [3]:
df_raw=pd.read_csv("../app/data/flight_data_raw.csv")

In [ ]:
df_raw.head()

Remove extreme outliers

In [ ]:
df=df_raw[(np.abs(df_raw.ARR_DELAY-df_raw.ARR_DELAY.mean()) <= (3*df_raw.ARR_DELAY.std())) | (df_raw["CANCELLED"]==1)]
# keep only the ones that are within +3 to -3 standard deviations in the column 'ARR_DELAY'.


In [ ]:
count=df.groupby(["ORIGIN","DEST"]).size()
count


In [ ]:
df[df["CANCELLED"]==1]

In [ ]:
#this works to get airport list
#df2 = df[["ORIGIN","DEST"]].groupby('ORIGIN').aggregate(lambda tdf: tdf.unique().tolist())

In [ ]:
df.shape

Atlanta has 166 Airports they fly to!

Create the final output csv:

Number of flights

In [ ]:
x = df.groupby(["ORIGIN","DEST"]).size()

idx = pd.MultiIndex.from_product((df.ORIGIN.unique(), df.DEST.unique()))

r = x.reindex(idx).dropna()
r=r.reset_index()



Average delay

In [ ]:

y = df.groupby(["ORIGIN","DEST"]).mean()["ARR_DELAY"]

idx = pd.MultiIndex.from_product((df.ORIGIN.unique(), df.DEST.unique()))

r_2 = y.reindex(idx).dropna()

r_2=r_2.reset_index()
#r_2.groupby('level_0').apply(lambda x: dict(zip(x['level_1'],x["ARR_DELAY"])))\

Causes of delay as percentage

In [ ]:

z = df.groupby(["ORIGIN","DEST"]).sum()[["CANCELLED","CARRIER_DELAY","WEATHER_DELAY","NAS_DELAY","SECURITY_DELAY","LATE_AIRCRAFT_DELAY"]]

r_3 = z.reindex(idx).dropna()

r_3=r_3.reset_index()
#r_2.groupby('level_0').apply(lambda x: dict(zip(x['level_1'],x["ARR_DELAY"])))\
r_3

In [ ]:
r_join= r.merge(r_3, how="left", on=["level_0","level_1"])
r_join= r_join.merge(r_2, how="left", on=["level_0","level_1"])
r_join.rename(columns= {0: "FLIGHT_VOLUME"}, inplace=True)

In [ ]:
r_join["VOLUME_RANK"]=r_join["FLIGHT_VOLUME"].rank(method='max',ascending=False).astype(int)

In [ ]:
r_join = r_join.rename(columns={"level_0": "origin", "level_1": "destination", "FLIGHT_VOLUME": "flight_volume","ARR_DELAY": "arr_delay",
                       "CANCELLED": "cancelled","CARRIER_DELAY":"carrier_delay","WEATHER_DELAY":"weather_delay","NAS_DELAY":"nas_delay",
                       "SECURITY_DELAY":"security_delay","LATE_AIRCRAFT_DELAY":"late_aircraft_delay","VOLUME_RANK":"volume_rank"})
r_join = r_join.sort_values(by=['origin'])

r_join.to_csv('flights_v1.csv', index=False)

In [ ]:
# target=r_join.groupby('level_0')[['level_1',"FLIGHT_VOLUME","ARR_DELAY","CANCELLED","CARRIER_DELAY","WEATHER_DELAY","NAS_DELAY","SECURITY_DELAY","LATE_AIRCRAFT_DELAY","VOLUME_RANK"]].apply(lambda x: x.set_index('level_1').to_dict(orient='index'))
# target
#

In [ ]:
# r
#

In [ ]:
# target.to_csv("airports_serviced_data.csv")
#